In [1]:
import gensim
from gensim.models import KeyedVectors
import numpy as np
import torch
from torch import nn, autograd, optim
import torch.nn.functional as F
import time
import math
from tqdm import tqdm

In [2]:
fvec = KeyedVectors.load_word2vec_format('vec_100d.txt', binary=False)
word_vec = fvec.vectors
vocab = ['<PAD>', '<BOS>', '<EOS>', '<UNK>']
vocab.extend(list(fvec.vocab.keys()))
word_vec = np.concatenate((np.array([[0]*word_vec.shape[1]] * 4), word_vec))
word_vec = torch.tensor(word_vec)

In [3]:
word_to_idx = {ch: i for i, ch in enumerate(vocab)}
idx_to_word = {i: ch for i, ch in enumerate(vocab)}

In [4]:
essays = []
topics = []
with open('composition.txt', 'r') as f:
    for line in f:
        essay, topic = line.replace('\n', '').split(' </d> ')
        essays.append(essay.split(' '))
        topics.append(topic.split(' '))

In [5]:
corpus_indice = list(map(lambda x: [word_to_idx[w] for w in x], essays[:8000]))
topics_indice = list(map(lambda x: [word_to_idx[w] for w in x], topics[:8000]))

In [6]:
length = list(map(lambda x: len(x), corpus_indice))

In [7]:
def tav_data_iterator(corpus_indice, topics_indice, batch_size, num_steps):
    epoch_size = len(corpus_indice) // batch_size
    for i in range(epoch_size):
        raw_data = corpus_indice[i*batch_size: (i+1)*batch_size]
        key_words = topics_indice[i*batch_size: (i+1)*batch_size]
        data = np.zeros((len(raw_data), num_steps+1), dtype=np.int64)
        for i in range(batch_size):
            doc = raw_data[i]
            tmp = [1]
            tmp.extend(doc)
            tmp.extend([2])
            tmp = np.array(tmp, dtype=np.int64)
            _size = tmp.shape[0]
            data[i][:_size] = tmp
        key_words = np.array(key_words, dtype=np.int64)
        x = data[:, 0:num_steps]
        y = data[:, 1:]
        mask = np.float32(x != 0)
        x = torch.tensor(x)
        y = torch.tensor(y)
        mask = torch.tensor(mask)
        key_words = torch.tensor(key_words)
        yield(x, y, mask, key_words)

In [8]:
class TATLSTM(nn.Module):
    def __init__(self, hidden_dim, embed_dim, num_layers, weight,
                 num_labels, bidirectional, dropout=0.5, **kwargs):
        super(TATLSTM, self).__init__(**kwargs)
        self.hidden_dim = hidden_dim
        self.embed_dim = embed_dim
        self.num_layers = num_layers
        self.num_labels = num_labels
        self.bidirectional = bidirectional
        if num_layers <= 1:
            self.dropout = 0
        else:
            self.dropout = dropout
        self.embedding = nn.Embedding.from_pretrained(weight)
        self.embedding.weight.requires_grad = False
        self.rnn = nn.GRU(input_size=self.embed_dim, hidden_size=self.hidden_dim,
                          num_layers=self.num_layers, bidirectional=self.bidirectional,
                          dropout=self.dropout)
        if self.bidirectional:
            self.decoder = nn.Linear(hidden_dim * 2 + self.embed_dim, self.num_labels)
        else:
            self.decoder = nn.Linear(hidden_dim + self.embed_dim, self.num_labels)
        self.attn = nn.Linear(self.embed_dim * 5, self.embed_dim)
        
        
    def forward(self, inputs, topics, hidden=None):
        embeddings = self.embedding(inputs)
        topics_embed = self.embedding(topics).float()
        topics_attn = self.attn(topics_embed.reshape((topics_embed.shape[0], -1)))
        topics_attn.unsqueeze_(-1)
#         for i in range(embeddings.shape[0]):
#             embeddings[i][0] = topics_embed[i]
        states, hidden = self.rnn(embeddings.permute([1, 0, 2]).float(), hidden)
        topics_attn = topics_attn.expand(topics_attn.shape[0], topics_attn.shape[1], states.shape[0])
        topics_attn = topics_attn.permute([2, 0, 1])
        states_with_topic = torch.cat([states, topics_attn], dim=2)
        outputs = self.decoder(states_with_topic.reshape((-1, states_with_topic.shape[-1])))
        return(outputs, hidden)
    
    def init_hidden(self, num_layers, batch_size, hidden_dim, **kwargs):
        hidden = torch.zeros(num_layers, batch_size, hidden_dim)
        return hidden

In [9]:
def predict_rnn(topics, num_chars, model, device, idx_to_word, word_to_idx):
    output = [1]
    topics = [word_to_idx[x] for x in topics]
    topics = torch.tensor(topics)
    topics = topics.reshape((1, topics.shape[0], -1))
    hidden = torch.zeros(num_layers, 1, hidden_dim)
    if use_gpu:
        hidden = hidden.to(device)
        topics = topics.to(device)
    for t in range(num_chars):
        X = torch.tensor(output[-1]).reshape((1, 1))
#         X = torch.tensor(output).reshape((1, len(output)))
        if use_gpu:
            X = X.to(device)
        pred, hidden = model(X, topics, hidden)
        if pred.argmax(dim=1)[-1] == 2:
            break
        else:
            output.append(int(pred.argmax(dim=1)))
#             output.append(int(pred.argmax(dim=1)[-1]))
    return(''.join([idx_to_word[i] for i in output[1:]]))

In [10]:
embedding_dim = 100
hidden_dim = 256
lr = 1e2
momentum = 0.0
num_epoch = 100
use_gpu = True
num_layers = 1
bidirectional = False
batch_size = 8
device = torch.device('cuda:1')
loss_function = nn.CrossEntropyLoss()

In [11]:
model = TATLSTM(hidden_dim=hidden_dim, embed_dim=embedding_dim, num_layers=num_layers,
                num_labels=len(vocab), weight=word_vec, bidirectional=bidirectional)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
if use_gpu:
#     model = nn.DataParallel(model)
    model.to(device)

In [12]:
predict_rnn(['妈妈', '希望', '长大', '孩子', '母爱'], 100, model, device, idx_to_word, word_to_idx)

'楚唐晓彤求子求子分肉智战子牙河楚唐晓彤求子求子分肉智战子牙河楚唐晓彤求子求子分肉智战子牙河楚唐晓彤求子求子分肉智战子牙河楚唐晓彤求子求子分肉智战子牙河楚唐晓彤求子求子分肉智战子牙河楚唐晓彤求子求子分肉智战子牙河楚唐晓彤求子求子分肉智战子牙河楚唐晓彤求子求子分肉智战子牙河楚唐晓彤求子求子分肉智战子牙河楚唐晓彤求子求子分肉智战子牙河楚唐晓彤求子求子分肉智战子牙河楚唐晓彤求子求子分肉智战子牙河楚唐晓彤求子求子分肉智战子牙河楚唐晓彤'

In [14]:
since = time.time()
for epoch in range(num_epoch):
    start = time.time()
    num, total_loss = 0, 0
#     if epoch == 5000:
#         optimizer.param_groups[0]['lr'] = lr * 0.1
    data = tav_data_iterator(corpus_indice, topics_indice, batch_size, max(length)+1)
    hidden = model.init_hidden(num_layers, batch_size, hidden_dim)
    weight = torch.ones(len(vocab))
    weight[0] = 0
    for X, Y, mask, topics in tqdm(data):
        num += 1
        hidden.detach_()
        if use_gpu:
            X = X.to(device)
            Y = Y.to(device)
            mask = mask.to(device)
            topics = topics.to(device)
            hidden = hidden.to(device)
            weight = weight.to(device)
        optimizer.zero_grad()
        output, hidden = model(X, topics, hidden)
        l = F.cross_entropy(output, Y.t().reshape((-1,)), weight)
        l.backward()
        norm = nn.utils.clip_grad_norm_(model.parameters(), 1e-2)
        optimizer.step()
        total_loss += l.item()
    end = time.time()
    s = end - since
    h = math.floor(s / 3600)
    m = s - h * 3600
    m = math.floor(m / 60)
    s -= (m * 60 + h * 3600)
    if(epoch % 10 == 0) or (epoch == (num_epoch - 1)):
        print('epoch %d/%d, loss %.4f, norm %.4f, time %.3fs, since %dh %dm %ds'
              %(epoch+1, num_epoch, total_loss / num, norm, end-start, h, m, s))
        print(predict_rnn(['妈妈', '希望', '长大', '孩子', '母爱'], 100, model, device, idx_to_word, word_to_idx))

1000it [01:27, 11.47it/s]


epoch 1/100, loss 7.8779, norm 0.9309, time 87.067s, since 0h 1m 27s


2it [00:00, 11.41it/s]

妈妈，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，我，


1000it [01:27, 11.40it/s]
1000it [01:27, 11.45it/s]
1000it [01:27, 11.44it/s]
1000it [01:27, 11.39it/s]
1000it [01:27, 11.39it/s]
1000it [01:27, 11.41it/s]
1000it [01:27, 11.39it/s]
1000it [01:27, 11.42it/s]
1000it [01:27, 11.41it/s]
1000it [01:27, 11.41it/s]


...

epoch 61/100, loss 2.5722, norm 1.4983, time 87.836s, since 1h 29m 16s
。


1000it [01:27, 11.41it/s]
1000it [01:27, 11.42it/s]
1000it [01:27, 11.33it/s]
1000it [01:27, 11.40it/s]
1000it [01:27, 11.36it/s]
1000it [01:27, 11.40it/s]
1000it [01:27, 11.35it/s]
1000it [01:27, 11.40it/s]
1000it [01:27, 11.41it/s]
1000it [01:27, 11.38it/s]
2it [00:00, 11.19it/s]

epoch 71/100, loss 2.5280, norm 1.5241, time 87.761s, since 1h 43m 54s
。


160it [00:14, 11.36it/s]

KeyboardInterrupt: 